In [ ]:
# New: To functions

import subprocess
import sys

# Install the 'requests' package if it is not already installed
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install('requests')

import pandas as pd
import requests

# Step 1: Download the Excel file from the URL
url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
response = requests.get(url)
with open('GSAF5.xls', 'wb') as file:
    file.write(response.content)

# Step 2: Load the Excel file
df_sa = pd.read_excel('GSAF5.xls')

# Step 3: Rename columns for better clarity and consistency
df_sa = df_sa.rename(columns={'Unnamed: 11': 'Fatal', 'Species ': 'Species', 'Source': 'Source', 'pdf': 'PDF'})

# Step 4: Clean the 'Fatal' column
df_sa['Fatal'] = df_sa['Fatal'].str.strip().str.upper().replace({
    'Y': 'Yes',
    'N': 'No',
    'F': 'Yes',
    'N N': 'No',
    'UNKNOWN': 'Unknown',
    'M': 'Unknown',  # Assuming 'M' means 'Unknown'
    'NQ': 'Unknown',  # Assuming 'NQ' means 'Unknown'
    'Y X 2': 'Yes'  # Assuming 'Y X 2' means 'Yes'
})
df_sa['Fatal'] = df_sa['Fatal'].fillna('Unknown')

# Step 5: Clean and standardize the 'Time' column
def standardize_time(time_str):
    if pd.isna(time_str):
        return '12:00'
    if isinstance(time_str, int):
        time_str = str(time_str)
    time_str = time_str.strip().lower()
    
    # Handle various time descriptions
    if 'early' in time_str or 'dawn' in time_str or 'before' in time_str:
        return '06:00'
    if 'morning' in time_str:
        return '09:00'
    if 'midday' in time_str or 'noon' in time_str:
        return '12:00'
    if 'afternoon' in time_str:
        return '15:00'
    if 'evening' in time_str or 'dusk' in time_str or 'sunset' in time_str:
        return '18:00'
    if 'night' in time_str or 'midnight' in time_str:
        return '23:00'
    
    # Try to parse times in different formats
    try:
        time_str = time_str.replace('h', ':').replace(' ', '')
        if '-' in time_str:
            time_str = time_str.split('-')[0].strip()
        if ':' in time_str:
            return pd.to_datetime(time_str, format='%H:%M', errors='coerce').strftime('%H:%M')
        if ' ' in time_str:
            time_str = time_str.split()[0]
        time_str = time_str.replace('j', '').replace('"', '').replace('pm', '').replace('am', '')
        if len(time_str) == 4:
            return f'{time_str[:2]}:{time_str[2:]}'
        if len(time_str) == 3:
            return f'0{time_str[0]}:{time_str[1:]}'
        return '12:00'  # Default value if parsing fails
    except:
        return '12:00'

df_sa['Time'] = df_sa['Time'].apply(standardize_time)

# Step 6: Clean the 'Species' column
valid_species = {
    'Tiger shark', 'White shark', 'Bull shark', 'Hammerhead shark', 'Great white shark', 
    'Mako shark', 'Blacktip shark', 'Reef shark', 'Nurse shark', 'Whale shark', 'Tiger shark'
}

def clean_species(species_str):
    if pd.isna(species_str):
        return 'Unknown'
    species_str = str(species_str).strip()
    # Extract only the main species name
    for name in valid_species:
        if name.lower() in species_str.lower():
            return name
    return 'Unknown'  # Default value if no valid species name is found

df_sa['Species'] = df_sa['Species'].apply(clean_species)

# Step 7: Clean the 'Source' column
df_sa['Source'] = df_sa['Source'].str.strip().replace({'UNKNOWN': 'Unknown'})

# Step 8: Clean the 'PDF' column
def clean_pdf(pdf_str):
    if pd.isna(pdf_str):
        return 'Unknown'
    if isinstance(pdf_str, int):
        pdf_str = str(pdf_str)
    pdf_str = str(pdf_str).strip()
    # Remove any non-alphanumeric characters except periods, underscores, and dashes
    pdf_str = ''.join(c for c in pdf_str if c.isalnum() or c in ['.', '_', '-'])
    return pdf_str if pdf_str else 'Unknown'

df_sa['PDF'] = df_sa['PDF'].apply(clean_pdf)

# Step 9: Save the cleaned data back to a new Excel file
df_sa.to_excel('Cleaned_GSAF5.xlsx', index=False)

print("Data cleaning complete. The cleaned data has been saved to 'Cleaned_GSAF5.xlsx'.")

# Step 10: Display the cleaned data
display(df_sa.head(100))



In [ ]:
# Old Main Code

import pandas as pd
from functions import load_data, clean_data, style_table

def main():
    # Load and clean data
    url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
    df_sa = load_data(url)
    df_sa = clean_data(df_sa)

    # Ask the user if they want to display the table
    show_table = input("Show table? Y/N: ").strip().lower()

    if show_table == 'y':
        styled_df = style_table(df_sa)
        display(styled_df)
    
    display(df_sa.head(10))

if __name__ == "__main__":
    main()

In [ ]:
# Show graphics for null values

import matplotlib.pyplot as plt
import seaborn as sns

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
df_sa = load_data(url)
df_sa = clean_data(df_sa)
sns.heatmap(df_sa.isnull(), cbar=False)
plt.show()